In [170]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.cluster import KMeans 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances
from sklearn.pipeline import Pipeline

import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

import os, sys, email,re


[nltk_data] Downloading package stopwords to /home/ki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ki/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [101]:
df = pd.read_csv('../projectfinder.csv')
df.dropna(axis=0, inplace=True)
df.head()
#df.describe()

,skill_summary,title,description
0,"tableau, visualisierung",Business Intelligence Analyst (m/w) - Tableau ...,Business Intelligence Analyst (m/w) - Tableau ...
1,", implementierung, java, ms sql, camel, apache...",Developer - Talend ESB oder Apache Camel (m/w)...,"Konzeption, Customizing sowie Softwareanpassun..."
2,", windows, clients, ios, mobile devices, mobil...",IT-Mitarbeiter (m/w) 1st / 2nd Level Support /...,"Als Mitglied eines kleinen, dynamischen Teams ..."
3,"dynamics ax 2009, dynamics crm 4.0, dynamics a...",Consultant für Microsoft Dynamics CRM (m/w),Über unseren Kunden: \n \n Unser Kunde ist e...
4,"hibernate, java, deutsch, oracle 11g, oracle 1...",Java Senior Developer - Backend (f/m),Über unseren Kunden: \n \n Unser Kunde ist e...


In [102]:
#split features
title = df['title'].tolist()
description = df['description'].tolist()
skill = df['skill_summary'].tolist()

In [103]:
#combine to make a lists of lists
all_feat = [title, description, skill]
features = ['title', 'description', 'skill']

In [104]:
#create new dataframe
new_df = pd.DataFrame(all_feat).transpose()
new_df.columns = features
new_df.head()

,title,description,skill
0,Business Intelligence Analyst (m/w) - Tableau ...,Business Intelligence Analyst (m/w) - Tableau ...,"tableau, visualisierung"
1,Developer - Talend ESB oder Apache Camel (m/w)...,"Konzeption, Customizing sowie Softwareanpassun...",", implementierung, java, ms sql, camel, apache..."
2,IT-Mitarbeiter (m/w) 1st / 2nd Level Support /...,"Als Mitglied eines kleinen, dynamischen Teams ...",", windows, clients, ios, mobile devices, mobil..."
3,Consultant für Microsoft Dynamics CRM (m/w),Über unseren Kunden: \n \n Unser Kunde ist e...,"dynamics ax 2009, dynamics crm 4.0, dynamics a..."
4,Java Senior Developer - Backend (f/m),Über unseren Kunden: \n \n Unser Kunde ist e...,"hibernate, java, deutsch, oracle 11g, oracle 1..."


In [173]:
# load nltk's German stopwords'
with open('../stopwords-de.txt', 'r') as f:
    stopwords = f.read().splitlines()
print(len(stopwords))
stopwords_eng = nltk.corpus.stopwords.words('english')
stopwords.extend(stopwords_eng)
stopwords.extend(['bewerben' 'direkt', 'melden', 'www', 'contactor'])
print(len(stopwords))

622
805


In [106]:
# load nltk's SnowballStemmer
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("german")

3


In [113]:
def isalph(mylist):
    return [item for item in mylist if item.isalpha()]
def remove_stop_words(mylist):
    return [item for item in mylist if item not in stopwords]
def lowercase(mylist):
    return [item.lower() for item in mylist]

In [119]:
#here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    #convert text list from list to a string
    #text_to_string = ','.join(str(v) for v in text)
    flatten_list = [item for sublist in text for item in sublist]
    # sentence tokenization and lowercasing first, then by word to ensure that punctuation is caught as it's own token
    list_of_tokens = [(nltk.word_tokenize(word)) for word in flatten_list]
    
    #extract only alphabetic tokens
    alphanumeric_tokens = [isalph(sub) for sub in list_of_tokens ]
    print(alphanumeric_tokens[:5])
    
    #lower case
    lowercased_list = [lowercase(word) for word in alphanumeric_tokens]
    #stopwords removal
    cleaned = [remove_stop_words(word) for word in lowercased_list]
    print(cleaned[:20])
    
    #stemming 
    #double for loop because its a lits of lits
    stemmed_tokens = [stemmer.stem(item) for token in cleaned for item in token]
    print(stemmed_tokens[:20])

    return stemmed_tokens

# this is used only for presentational purposes
def tokenize_only(text):
    #convert text list from list to a string
    #text_to_string = ','.join(str(v) for v in text)
    flatten_list = [item for sublist in text for item in sublist]
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    list_of_tokens = [(nltk.word_tokenize(word)) for word in flatten_list]
    #lower case
    lowercased_list = [lowercase(word) for word in list_of_tokens]
    #stopwords removal
    cleaned = [remove_stop_words(word) for word in lowercased_list]
    print(cleaned[:20])
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    filtered_tokens = [isalph(sub) for sub in cleaned ]
    

    return filtered_tokens

In [120]:
stemmed_text = tokenize_and_stem(all_feat)
tokenized_text = tokenize_only(all_feat)
#for word in description:
 #   stemmed_word = tokenize_and_stem(word) #for each item in 'synopses', tokenize/stem
  #  all_tokenized = tokenize_only(word)
   # stemmed_text.extend(stemmed_word) #extend the 'totalvocab_stemmed' list
    #tokenized_text.extend(all_tokenized)



[['Business', 'Intelligence', 'Analyst', 'Tableau', 'Desktop', 'FFM'], ['Developer', 'Talend', 'ESB', 'oder', 'Apache', 'Camel'], ['Level', 'Support', 'Onsite', 'Support'], ['Consultant', 'für', 'Microsoft', 'Dynamics', 'CRM'], ['Java', 'Senior', 'Developer', 'Backend']]
[['business', 'intelligence', 'analyst', 'tableau', 'desktop', 'ffm'], ['developer', 'talend', 'esb', 'apache', 'camel'], ['level', 'support', 'onsite', 'support'], ['consultant', 'microsoft', 'dynamics', 'crm'], ['java', 'senior', 'developer', 'backend'], ['dms', 'cms', 'consultant', 'prozeßerfahrung'], ['digitalisierung', 'galvanik', 'endkunde'], ['ux', 'designer', 'web', 'app'], ['linux', 'systemadministrator'], ['requirements', 'engineer', 'wertpapiere'], ['developer'], ['java', 'entwickler'], ['system', 'engineer', 'schwerpunkt', 'ms', 'exchange', 'active', 'directory'], ['sap', 'technical', 'consultant', 'sap', 'entwickler'], ['inhouse', 'prozessmanager', 'dynamics', 'nav'], ['business', 'intelligence', 'consulta

In [158]:
combined_2 = df['title'] + ' '  + df['description'] + ' '  + df['skill_summary']

In [ ]:
true_k = 5
pipeline = Pipeline([
    ('vectorizer',  CountVectorizer(ngram_range=(1, 3), stop_words=stopwords)),
    ('clusterer',  KMeans(n_clusters=true_k)) ])

pipeline.fit(combined_2)

In [ ]:
pipeline.predict(examples)

In [166]:
vector = CountVectorizer(ngram_range=(1, 3), stop_words=stopwords)

In [167]:
X = vector.fit_transform(combined_2)
print(X.shape)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['wahr'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(5360, 603175)


In [169]:
true_k = 5
model = KMeans(n_clusters=true_k)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [171]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vector.get_feature_names()

In [172]:
for i in range(true_k):
    print()
    print(f'Cluster {i}')
    
    for ind in order_centroids[i, :10]:
        print(terms[ind])


Cluster 0
sap
bewerben
direkt
melden
bewerben direkt
direkt bewerben
melden bewerben
bewerben direkt bewerben
melden bewerben direkt
abap

Cluster 1
bewerben
direkt
melden
direkt bewerben
bewerben direkt
melden bewerben direkt
melden bewerben
bewerben direkt bewerben
java
kenntnisse

Cluster 2
contractor
bewerben
https
frankfurt
https www contractor
https www
contractor de
de
www contractor de
www contractor

Cluster 3
contractor
https
bewerben
münchen
www contractor de
www
https www contractor
https www
www contractor
contractor de

Cluster 4
bewerben
daten
cv
bewerben direkt bewerben
direkt
direkt bewerben
melden bewerben
bewerben direkt
melden bewerben direkt
melden


In [127]:

print("Prediction")
X = vectorizer.transform(["Produktumfeld der Firma VMWARE: Airwatch (sehr gute Kenntnisse)Netzwerktechnik: LAN, DMZ, Rechenzentrum (Vertiefte Kenntnisse) MS-Office Tools: Word, Excel, Visio, Powerpoint (Vertiefte Kenntnisse) Bereitschaft zur Sicherheitsüberprüfung Level 2 (Ü2)"])
predicted = model.predict(X)
print(predicted)

Prediction
[0]


In [142]:
test_data = [['Konzeption, Customizing sowie Softwareanpassungen mit Talend ESB  Implementierung von Softwaresystemen mit Java  Analyse sowie Design von Softwarearchitekturen'], ['Developer - Talend ESB oder Apache Camel (m/w) – Schleswig-Holstein #3076'],["mybatis","datenbanken","analyse","talend","softwarearchitektur","java","sql","datenmodelle","ms sql","requirements","apache camel","testautomatisierung","ms","webservices","oracle","entwicklung","esb","apache","camel","implementierung"]]

In [144]:
vec = vectorizer.transform(test_data)
predicted = model.predict(vec)
print(predicted)

AttributeError: 'list' object has no attribute 'lower'